大型语言模型（LLM，如BERT、GPT等）处理文本数据的一般流程。这个过程可以细分为以下几个步骤：

1. **分词（Tokenization）**：
   - 这一步骤是将文本字符串分解成较小的单位，称为“词元”（tokens）。分词可以根据不同的规则进行，如基于空格和标点符号分割、子词分割（如Byte-Pair Encoding, BPE）或使用更复杂的基于规则或学习的方法。

2. **词元到ID的映射（Token to ID mapping）**：
   - 分词后，每个词元会被转换成一个唯一的数字ID。这些ID对应于语言模型的词汇表中的索引。这一步是必要的，因为计算机模型不能直接处理文本数据，而是处理数字。

3. **嵌入层（Embedding）**：
   - 在将词元转换为ID之后，这些ID将被用来在嵌入层中查找每个词元的向量表示。嵌入层是一个可训练的参数矩阵，模型通过学习来调整这些向量，以便更好地捕捉词义和语法关系。
   - 这些嵌入通常包括词嵌入，可能还包括位置嵌入（指示词元在句子中的位置）和分段嵌入（特别是在处理多个句子时）。

4. **模型处理（Model Processing）**：
   - 嵌入向量随后会被送入语言模型的主体，如Transformer结构。这些模型会通过一系列的层处理这些向量，每一层都包括例如自注意力机制和前馈神经网络等组件。模型通过这种方式来理解和生成文本上下文的表示。

5. **输出解析（Output Decoding）**：
   - 对于生成任务，如文本续写，模型的输出通常是在词汇表上的概率分布，这些概率表明下一个词元是什么。然后可以使用不同的策略（如贪婪搜索、束搜索）从这些概率中选择词元来生成文本。
   - 对于分类任务（如情感分析），模型输出通常是分类标签的概率。

6. **后处理（Post-processing）**：
   - 在文本生成任务中，输出的词元序列通常会经过后处理转换回可读的文本。这可能包括去除特殊词元、调整格式等步骤。



# Dataset

In [2]:
import datasets

In [5]:
# https://huggingface.co/datasets/Helsinki-NLP/opus_books/viewer/en-it

from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus_books", 'en-it')


Generating train split: 100%|██████████| 32332/32332 [00:00<00:00, 610329.07 examples/s]


In [7]:
for i in range(9):
    print(dataset['train'][i])

{'id': '0', 'translation': {'en': 'Source: Project Gutenberg', 'it': 'Source: www.liberliber.it/Audiobook available here'}}
{'id': '1', 'translation': {'en': 'Jane Eyre', 'it': 'Jane Eyre'}}
{'id': '2', 'translation': {'en': 'Charlotte Bronte', 'it': 'Charlotte Brontë'}}
{'id': '3', 'translation': {'en': 'CHAPTER I', 'it': 'PARTE PRIMA'}}
{'id': '4', 'translation': {'en': 'There was no possibility of taking a walk that day.', 'it': 'I. In quel giorno era impossibile passeggiare.'}}
{'id': '5', 'translation': {'en': 'We had been wandering, indeed, in the leafless shrubbery an hour in the morning; but since dinner (Mrs. Reed, when there was no company, dined early) the cold winter wind had brought with it clouds so sombre, and a rain so penetrating, that further out-door exercise was now out of the question.', 'it': "La mattina avevamo errato per un'ora nel boschetto spogliato di foglie, ma dopo pranzo (quando non vi erano invitati, la signora Reed desinava presto), il vento gelato d'inv

# Tokenizer

In [2]:
import torch 
import torch.nn as nn
from pathlib import Path
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
config = {
    'datasource': 'opus_books',
    'lang_src': 'en',
    'lang_tgt': 'it',
    'tokenizer_file': 'tokenizer_{0}.json'
    }

In [9]:
def get_all_sentences(dataset, language):
    for item in dataset:
        yield item['translation'][language]

def get_or_build_tokenizer(config, dataset, language):
    tokenizer_path = Path(config['tokenizer_file'].format(language))
    if not tokenizer_path.exists():
        tokenizer = Tokenizer(WordLevel(unk_token='[UNK]'))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(dataset, language), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    
    return tokenizer

In [10]:
# It only has the train split, so we divide it overselves
dataset_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, dataset_raw, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, dataset_raw, config['lang_tgt'])

In [14]:
# Encode a sample sentence
encoding = tokenizer_src.encode("I love a cat.")

# Print the tokens
print("Tokens:", encoding.tokens) # 提供了分词后的词元列表
# Print the token IDs
print("Token IDs:", encoding.ids) # 提供了每个词元对应的在分词器词典中的索引ID。

Tokens: ['I', 'love', 'a', 'cat', '.']
Token IDs: [9, 194, 11, 1812, 7]


In [21]:
config = {
    'datasource': 'Helsinki-NLP/opus-100',
    'lang_src': 'en',
    'lang_tgt': 'zh',
    'tokenizer_file': 'tokenizer_{0}.json'
    }

In [22]:
# It only has the train split, so we divide it overselves
dataset_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, dataset_raw, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, dataset_raw, config['lang_tgt'])

Generating validation split: 100%|██████████| 2000/2000 [00:00<00:00, 454963.01 examples/s]


In [28]:
# Encode a sample sentence
encoding = tokenizer_tgt.encode("阿拉伯国家联盟首脑级理事会")

# Print the tokens
print("Tokens:", encoding.tokens) # 提供了分词后的词元列表
# Print the token IDs
print("Token IDs:", encoding.ids) # 提供了每个词元对应的在分词器词典中的索引ID。

Tokens: ['阿拉伯国家联盟首脑级理事会']
Token IDs: [8294]
